CS641 Assignment 5

In [ ]:
from pyfinite import ffield 
import numpy as np
import random

In [ ]:
#charcters used in output are from f to u, so f is 0000 and so on 16 values
bin_char = {'0000': 'f','0001': 'g','0010': 'h','0011': 'i','0100': 'j','0101': 'k','0110': 'l','0111': 'm','1000': 'n','1001': 'o','1010': 'p','1011': 'q','1100': 'r','1101': 's','1110': 't','1111': 'u'}
F = ffield.FField(7) #2^7 element field
plain = open("plaintext.txt", 'r')
cipher = open("ciphertext.txt", 'r')

In [ ]:
def Fexp(b,e):  #Exponential function in fields
    result = 1
    for _ in range(e):
        result = F.Multiply(result,b)
    return result

In [ ]:
#blocks to hex values
def block_to_byte(block):
    result = ""
    for i in range(0, len(block), 2):
        byte = block[i:i+2]
        char = chr(16 * (ord(byte[0]) - ord('f')) + ord(byte[1]) - ord('f'))
        result += char
    return result

In [ ]:
#similar to above, takes hex values and depending on flag adds first or second part to the result
def hex_values(input_line, index, flag):
    result = []
    for hex_value in input_line.strip().split(" "):
        temp_string = ""
        for i in range(0, len(hex_value), 2):
            byte = hex_value[i:i+2]
            char = chr(16 * (ord(byte[0]) - ord('f')) + ord(byte[1]) - ord('f'))
            temp_string += char
        if flag:
            result.append(temp_string[index+1])
        else:
            result.append(temp_string[index])
    return result

In [ ]:
def cryptanalysis(plaintext, lin_mat, exp_mat):
    # Convert plaintext characters to their ASCII codes
    plaintext = [ord(c) for c in plaintext]
    
    # Initialize output matrix
    Output = [[0 for j in range (8)] for i in range(8)]
    
    # Apply Fexp function to each element of plaintext using corresponding element from exp_mat
    for index, elem in enumerate(plaintext):
        Output[0][index] = Fexp(elem, exp_mat[index])
    
    # Perform linear transformation on lin_mat and Output[0]
    mat = lin_mat
    elist = Output[0]
    result = [0, 0, 0, 0, 0, 0, 0, 0]
    for row, elem in zip(mat, elist):
        temp_res = [0, 0, 0, 0, 0, 0, 0, 0]
        for i, e in enumerate(row):
            temp_res[i] = F.Multiply(e, elem)
        temp_result = [0, 0, 0, 0, 0, 0, 0, 0]
        for i, (e1, e2) in enumerate(zip(temp_res, result)):
            temp_result[i] = int(e1) ^ int(e2)
        result = temp_result
    Output[1] = result
    
    # Apply Fexp function to each element of Output[1] using corresponding element from exp_mat
    for index, elem in enumerate(Output[1]):
        Output[2][index] = Fexp(elem, exp_mat[index])
    
    # Perform linear transformation on lin_mat and Output[2]
    mat = lin_mat
    elist = Output[2]
    result = [0,0,0,0,0,0,0,0]
    for row, elem in zip(mat, elist):
        temp_res = [0,0,0,0,0,0,0,0]
        for i, e in enumerate(row):
            temp_res[i] = F.Multiply(e, elem)
        temp_result = [0,0,0,0,0,0,0,0]
        for i, (e1, e2) in enumerate(zip(temp_res, result)):
            temp_result[i] = int(e1) ^ int(e2)
        result = temp_result
    Output[3] = result

    # Apply Fexp function to each element of Output[1] using corresponding element from exp_mat
    for ind, elem in enumerate(Output[3]):
        Output[4][ind] = Fexp(elem, exp_mat[ind])
        
    return Output[4]

In [ ]:
def find_block_values():
    #store possible values for exponentiation and Linear transformation function
    E = [[] for _ in range(8)]
    A = [[[] for _ in range(8)] for _ in range(8)]
    
    for index, (i, o) in enumerate(zip(plain.readlines(), cipher.readlines())):
        #convert into hex
        i_hex = hex_values(i, index, False)
        o_hex = hex_values(o, index, False)
        for i in range(1, 127):
            for j in range(1, 128):
                flag = True
                for input_value, output_value in zip(i_hex, o_hex):
                    temp1 = Fexp(ord(input_value), i)
                    temp2 = Fexp(F.Multiply(temp1, j), i)
                    temp3 = Fexp(F.Multiply(temp2, j), i)
                    if ord(output_value) != temp3:
                        flag = False
                        break
                if flag:
                    E[index].append(i)
                    A[index][index].append(j)
    return A, E
A,E = find_block_values()

In [ ]:
A

In [ ]:
E

In [ ]:
def find_matrix():
    for index, (input_line, output_line) in enumerate(zip(plain.readlines(), cipher.readlines())):
        if index > 6:
            break
        input_hex_values = hex_values(input_line, index, False)
        output_hex_values = hex_values(output_line, index, True)
            
        for i in range(1, 128):
            for p1, e1 in zip(E[index + 1], A[index + 1][index + 1]):
                for p2, e2 in zip(E[index], A[index][index]):
                    flag = True
                    for input_value, output_value in zip(input_hex_values, output_hex_values):
                        temp1 = Fexp(ord(input_value), p2)
                        temp2 = F.Multiply(temp1, e2)
                        temp3 = Fexp(temp2, p2)
                        temp4 = F.Multiply(temp3, i) 
                        temp5 = F.Multiply(Fexp(F.Multiply(Fexp(ord(input_value), p2), i), p1), e1)
                        if ord(output_value) != Fexp(int(temp4) ^ int(temp5), p1):
                            flag = False
                            break
                    if flag:
                        E[index + 1] = [p1]
                        A[index + 1][index + 1] = [e1]
                        E[index] = [p2]
                        A[index][index] = [e2]
                        A[index][index + 1] = [i]
    return A, E


In [ ]:
def final_mv(A, E):
    for index in range(6):
        offset = index + 2
        exp_list = [e[0] for e in E]
        lin_trans_list = [[0 for i in range(8)] for j in range(8)]
        for i in range(8):
            for j in range(8):
                lin_trans_list[i][j] = 0 if len(A[i][j]) == 0 else A[i][j][0]
        
        for ind, (input_line, output_line) in enumerate(zip(plain.readlines(), cipher.readlines())):
            if ind > (7 - offset):
                continue
            input_values = []
            for msg in input_line.strip().split(" "):
                input_values.append(block_to_byte(msg))
            output_values = []
            for msg in output_line.strip().split(" "):
                output_values.append(block_to_byte(msg))
         
            for i in range(1, 128):
                lin_trans_list[ind][ind + offset] = i
                flag = True
                for input_value, output_value in zip(input_values, output_values):
                    if cryptanalysis(input_value, lin_trans_list, exp_list)[ind + offset] != ord(output_value[ind + offset]):
                        flag = False
                        break
                if flag:
                    A[ind][ind + offset] = [i]
    return A, exp_list

In [ ]:
A,E=find_matrix()
A,E=final_mv(A,E)
lin_trans_list = [[0 for i in range(8)] for j in range(8)]
for i in range(8):
    for j in range(8):
        lin_trans_list[i][j] = 0 if len(A[i][j]) == 0 else A[i][j][0]

In [ ]:
lin_trans_list

In [ ]:
def DecryptPassword(password):
    passw = block_to_byte(password)
    result = ""
    for index in range(8):
        for ans in range(128):
            temp1 = '{:0>8}'.format(format(ans, "b"))
            tempa = bin_char[temp1[0:4]]
            tempb = bin_char[temp1[4:8]]
            inp = result + tempa + tempb + (16 - len(result) - 2) * 'f'
            if ord(passw[index]) == cryptanalysis(block_to_byte(inp), lin_trans_list, E)[index]:
                b = ans
                temp1 = '{:0>8}'.format(format(b, "b"))
                a = bin_char[temp1[0:4]], bin_char[temp1[4:8]]
                result += a[0] + a[1]
                break
    result = block_to_byte(result)
    return result

In [ ]:
#ciphertext password in two blocks of 8 bytes
c1 = "jghrlgmnhsgfffjk"
c2 = "gkikhrmjkrgigjmu"
#plaintexts corresponding to them
p1 = DecryptPassword(c1)
p2 = DecryptPassword(c2)
#printing the plaintext password
print(p1+p2)